In [180]:
import tensorflow as tf
import os
import cv2
from matplotlib import pyplot as plt
%matplotlib inline  

from datasets import flower_dataset
#from models import models
from backbone import inception
from preprocessing import inception_preprocessing

In [181]:
dataset_dir = '/home/dan/prj/datasets/flower_exp1/train'

In [182]:
def load_categorical_data(dataset_dir):
  filepaths = {}
  class_names = []
  for dir_name in os.listdir(dataset_dir):
    directory = os.path.join(dataset_dir, dir_name)
    if os.path.isdir(directory):
        filepaths[dir_name] = []
        for filename in os.listdir(directory):
            path = os.path.join(directory, filename)
            filepaths[dir_name].append(path)
        class_names.append(dir_name)
  class_names_to_ids = dict(zip(class_names, range(len(class_names))))
  return filepaths, class_names_to_ids

In [183]:
def _parse_function(filename, label=None):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string, channels=3)
  # image_resized = tf.image.resize_images(image_decoded, [224, 224], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  images = inception_preprocessing.preprocess_image(
      image_decoded, height=224, width=224, is_training=True)

  return images

In [184]:
def categorical_input_fn(filepaths, class_names_to_ids, categorical_batch_size, num_images, mode="training"):
    data = {}
    for name in class_names_to_ids.keys():
        dataset = tf.data.Dataset.from_tensor_slices(tf.cast(
            filepaths[name], tf.string))
        dataset = dataset.shuffle(len(filepaths[name]))
        dataset = dataset.repeat()
        dataset = dataset.map(_parse_function, num_parallel_calls=2)
        dataset = dataset.batch(categorical_batch_size)
        dataset = dataset.prefetch(2 * batch_size)
        iterator = dataset.make_one_shot_iterator()
        data[name] = iterator.get_next()

In [185]:
filepaths, class_names_to_ids = load_categorical_data(dataset_dir)

In [186]:
batch_size = 10

In [189]:
data = {}
for name in class_names_to_ids.keys():
    dataset_filepath = tf.data.Dataset.from_tensor_slices(tf.cast(
        filepaths[name], tf.string))
    dataset_class = tf.data.Dataset.from_tensor_slices(
        [class_names_to_ids[os.path.basename(os.path.dirname(filepath))] for filepath in filepaths[name]])

    dataset = tf.data.Dataset.zip((dataset_filepath, dataset_class))
    dataset = dataset.shuffle(len(filepaths[name]))
    dataset = dataset.repeat()
    #dataset = dataset.map(_parse_function, num_parallel_calls=2)
    dataset = dataset.batch(1)
    dataset = dataset.prefetch(2 * batch_size)
    iterator = dataset.make_one_shot_iterator()
    data[name] = iterator.get_next()

In [190]:
#stacked_data = tf.stack([x for x in data.values()])
#stacked_data = tf.reshape(stacked_data, shape=(-1, 224, 224, 3))

In [194]:
stacked_data = tf.stack([x for x in data.values()])

TypeError: Cannot convert a list containing a tensor of dtype <dtype: 'int32'> to <dtype: 'string'> (Tensor is: <tf.Tensor 'IteratorGetNext_76:1' shape=(?,) dtype=int32>)

In [191]:
sess = tf.Session()
x = sess.run(data['roses'])

In [192]:
x

(array([b'/home/dan/prj/datasets/flower_exp1/train/roses/8209458141_38f38be65c_m.jpg'],
       dtype=object), array([0], dtype=int32))

In [170]:
with tf.Session() as sess:
    result = sess.run(stacked_data)

In [175]:
result[0]

array([[[[-0.6800072 , -0.8124637 , -0.9449202 ],
         [-0.689391  , -0.79375964, -0.95593005],
         [-0.7080185 , -0.78607315, -0.9812099 ],
         ...,
         [-0.6855675 , -0.61446077, -0.97339433],
         [-0.71319807, -0.6519962 , -0.9914158 ],
         [-0.73731124, -0.68322825, -1.        ]],

        [[-0.6800072 , -0.8124637 , -0.9449202 ],
         [-0.689391  , -0.79375964, -0.95593005],
         [-0.7049372 , -0.7829919 , -0.9781286 ],
         ...,
         [-0.6654031 , -0.59089637, -0.9566736 ],
         [-0.6979122 , -0.6284318 , -0.9743559 ],
         [-0.7129092 , -0.65333325, -0.9927529 ]],

        [[-0.6800072 , -0.8124637 , -0.9449202 ],
         [-0.689391  , -0.79375964, -0.95593005],
         [-0.7048547 , -0.7829965 , -0.9781042 ],
         ...,
         [-0.6574401 , -0.58297694, -0.9586154 ],
         [-0.6899939 , -0.6204686 , -0.9665405 ],
         [-0.6987469 , -0.63909703, -0.97854567]],

        ...,

        [[ 0.3235991 , -0.03215575, -0

In [122]:
data.values()

dict_values([<tf.Tensor 'IteratorGetNext_36:0' shape=(?, 224, 224, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_37:0' shape=(?, 224, 224, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_38:0' shape=(?, 224, 224, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_39:0' shape=(?, 224, 224, 3) dtype=float32>, <tf.Tensor 'IteratorGetNext_40:0' shape=(?, 224, 224, 3) dtype=float32>])

In [123]:
data['roses'].shape

TensorShape([Dimension(None), Dimension(224), Dimension(224), Dimension(3)])

In [124]:
stacked_data = [x for x in data.values()]

In [126]:
stacked_data

[<tf.Tensor 'IteratorGetNext_36:0' shape=(?, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'IteratorGetNext_37:0' shape=(?, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'IteratorGetNext_38:0' shape=(?, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'IteratorGetNext_39:0' shape=(?, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'IteratorGetNext_40:0' shape=(?, 224, 224, 3) dtype=float32>]

In [132]:
help(tf.reshape)

Help on function reshape in module tensorflow.python.ops.gen_array_ops:

reshape(tensor, shape, name=None)
    Reshapes a tensor.
    
    Given `tensor`, this operation returns a tensor that has the same values
    as `tensor` with shape `shape`.
    
    If one component of `shape` is the special value -1, the size of that dimension
    is computed so that the total size remains constant.  In particular, a `shape`
    of `[-1]` flattens into 1-D.  At most one component of `shape` can be -1.
    
    If `shape` is 1-D or higher, then the operation returns a tensor with shape
    `shape` filled with the values of `tensor`. In this case, the number of elements
    implied by `shape` must be the same as the number of elements in `tensor`.
    
    For example:
    
    ```
    # tensor 't' is [1, 2, 3, 4, 5, 6, 7, 8, 9]
    # tensor 't' has shape [9]
    reshape(t, [3, 3]) ==> [[1, 2, 3],
                            [4, 5, 6],
                            [7, 8, 9]]
    
    # tensor 't' 

In [133]:
real_stacked_data = tf.stack(stacked_data)
new_real_stacked_data = tf.reshape(real_stacked_data, shape=(-1, 224, 224, 3))

In [134]:
with tf.Session() as sess:
    result = sess.run(new_real_stacked_data)

In [135]:
result.shape

(50, 224, 224, 3)

In [54]:
with tf.Session() as sess:
    real_images = sess.run(images)

In [56]:
real_images.shape

(10, 224, 224, 3)

In [68]:
dataset1 = tf.data.Dataset.from_tensor_slices( list(range(10)) )
print("dataset1 : ")
print(dataset1.output_types)
print(dataset1.output_shapes)

dataset2 = tf.data.Dataset.from_tensor_slices( list(range(10)))

print("dataset2 : ")
print(dataset2.output_types)
print(dataset2.output_shapes)

dataset3 = dataset1.concatenate(dataset2)

dataset1 : 
<dtype: 'int32'>
()
dataset2 : 
<dtype: 'int32'>
()


In [75]:
x = [s1, s2]

In [76]:
x

[<tf.Tensor 'Const:0' shape=(3,) dtype=int32>,
 <tf.Tensor 'Const_1:0' shape=(3,) dtype=int32>]

In [80]:
x = tf.placeholder(shape=[None, 224, 224, 3], dtype = tf.int32)
y = tf.placeholder(shape=[None, 224, 224, 3], dtype = tf.int32)

s = [x, y]

print(s)

[<tf.Tensor 'Placeholder:0' shape=(?, 224, 224, 3) dtype=int32>, <tf.Tensor 'Placeholder_1:0' shape=(?, 224, 224, 3) dtype=int32>]


In [81]:
new = tf.stack(x)

In [83]:
new

<tf.Tensor 'Placeholder:0' shape=(?, 224, 224, 3) dtype=int32>